# Importing libraries

In [1]:
import os
import sys
from dotenv import load_dotenv
from typing import Tuple
import torch
from torch.utils.data import Dataset, DataLoader
import wandb
from src.utils import set_seed, load_text, split_text, speedometer
from src.config import ModelConfig, TrainConfig, GenerationConfig
from src.tokenizer import CharTokenizer
from models.GPT import GPT
from src.train import Trainer

In [2]:
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(PROJECT_ROOT)
print(f"PROJECT_ROOT: {PROJECT_ROOT}")

PROJECT_ROOT: /workspace/PathFinder


# Configuration

In [3]:
model_config = ModelConfig(
    vocab_size=-1,
    max_seq_len=128,
    d_embed=128,
    n_layers=4,
    n_heads=4,
    d_head=32,
    d_ff=512,
    rank=32
)

train_config = TrainConfig(
    debug=True,
    wandb_project="nanoGPT",
    model_name="nanoGPT",
    per_device_train_batch_size=512,
    per_device_eval_batch_size=1024,
    gradient_accumulation_steps=512 // 512,
    num_train_epochs=1,
    learning_rate=5e-3,
    eval_steps=100,
    mixed_precision=True,
    matmul_precision="high",
)

generation_config = GenerationConfig(
    use_cache=True,
    max_new_tokens=1000,
    temperature=1.0,
    top_k=50
)

In [4]:
load_dotenv()
wandb.login(key=os.environ.get("WANDB_API_KEY"))

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pathfinderkr to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# Utils

## Reproducibility

In [5]:
set_seed(train_config.seed)

Random seed set to 42


## Device

In [6]:
device = torch.device("cuda")
print(f"Device: {torch.cuda.get_device_name(device)}")
torch.set_float32_matmul_precision(train_config.matmul_precision)  # Tensor Cores
print(f"MatMul Precision: {train_config.matmul_precision}")

Device: NVIDIA RTX A6000
MatMul Precision: high


# Dataset

In [7]:
dataset_path = os.path.join(PROJECT_ROOT, "datasets/Shakespeare/shakespeare.txt")
shakespeare_text = load_text(dataset_path)

Loaded text data from /workspace/PathFinder/datasets/Shakespeare/shakespeare.txt (length: 1115394 characters).


In [8]:
if train_config.debug:
    subset_shakespeare_text = shakespeare_text[:10000]
    print(subset_shakespeare_text)
    shakespeare_text = subset_shakespeare_text

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



# Tokenizer

In [9]:
char_tokenizer = CharTokenizer()
char_tokenizer.build_vocab(text=shakespeare_text)
vocab_path = os.path.join(PROJECT_ROOT, "datasets/Shakespeare/vocab.json")
char_tokenizer.save_vocab(vocab_path)
model_config.vocab_size = char_tokenizer.vocab_size

Vocabulary size: 61
Vocabulary saved to /workspace/PathFinder/datasets/Shakespeare/vocab.json.


In [10]:
if train_config.debug:
    print("Vocabulary:", char_tokenizer.char2idx)

Vocabulary: {'<|begin_of_text|>': 0, '<|end_of_text|>': 1, '<|UNK|>': 2, '<|PAD|>': 3, '\n': 4, ' ': 5, '!': 6, "'": 7, ',': 8, '-': 9, '.': 10, ':': 11, ';': 12, '?': 13, 'A': 14, 'B': 15, 'C': 16, 'D': 17, 'E': 18, 'F': 19, 'H': 20, 'I': 21, 'J': 22, 'L': 23, 'M': 24, 'N': 25, 'O': 26, 'P': 27, 'R': 28, 'S': 29, 'T': 30, 'U': 31, 'V': 32, 'W': 33, 'Y': 34, 'a': 35, 'b': 36, 'c': 37, 'd': 38, 'e': 39, 'f': 40, 'g': 41, 'h': 42, 'i': 43, 'j': 44, 'k': 45, 'l': 46, 'm': 47, 'n': 48, 'o': 49, 'p': 50, 'q': 51, 'r': 52, 's': 53, 't': 54, 'u': 55, 'v': 56, 'w': 57, 'x': 58, 'y': 59, 'z': 60}


# Preprocessing

In [11]:
train_text, val_text = split_text(shakespeare_text, val_size=0.1)
print(f"Training text length: {len(train_text)} characters")
print(f"Validation text length: {len(val_text)} characters")

Training text length: 9000 characters
Validation text length: 1000 characters


In [12]:
class TextDataset(Dataset):
    def __init__(self, text: str, tokenizer: CharTokenizer, max_seq_len: int):
        self.encoded = tokenizer.encode(text)
        self.max_seq_len = max_seq_len

    def __len__(self) -> int:
        return len(self.encoded) - self.max_seq_len

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        input_ids = self.encoded[idx:idx + self.max_seq_len]
        target_ids = self.encoded[idx + 1:idx + self.max_seq_len + 1]
        return input_ids, target_ids

def collate_fn(batch):
    input_ids = torch.stack([item[0] for item in batch])
    target_ids = torch.stack([item[1] for item in batch])
    return {"input_ids": input_ids, "target_ids": target_ids}

train_dataset = TextDataset(train_text, char_tokenizer, model_config.max_seq_len)
val_dataset = TextDataset(val_text, char_tokenizer, model_config.max_seq_len)

train_loader = DataLoader(
    train_dataset,
    collate_fn=collate_fn,
    batch_size=train_config.per_device_eval_batch_size,
    shuffle=True,
    num_workers=4
)
val_loader = DataLoader(
    val_dataset,
    collate_fn=collate_fn,
    batch_size=train_config.per_device_eval_batch_size,
    shuffle=False,
    num_workers=4
)

In [13]:
if train_config.debug:
    sample_batch = next(iter(train_loader))
    print(f"Sample input IDs: {sample_batch['input_ids'][0]}")
    print(f"Sample target IDs: {sample_batch['target_ids'][0]}")

Sample input IDs: tensor([10,  4,  4, 14, 46, 46, 11,  4, 33, 39,  5, 45, 48, 49, 57,  7, 54,  8,
         5, 57, 39,  5, 45, 48, 49, 57,  7, 54, 10,  4,  4, 19, 43, 52, 53, 54,
         5, 16, 43, 54, 43, 60, 39, 48, 11,  4, 23, 39, 54,  5, 55, 53,  5, 45,
        43, 46, 46,  5, 42, 43, 47,  8,  5, 35, 48, 38,  5, 57, 39,  7, 46, 46,
         5, 42, 35, 56, 39,  5, 37, 49, 52, 48,  5, 35, 54,  5, 49, 55, 52,  5,
        49, 57, 48,  5, 50, 52, 43, 37, 39, 10,  4, 21, 53,  7, 54,  5, 35,  5,
        56, 39, 52, 38, 43, 37, 54, 13,  4,  4, 14, 46, 46, 11,  4, 25, 49,  5,
        47, 49])
Sample target IDs: tensor([ 4,  4, 14, 46, 46, 11,  4, 33, 39,  5, 45, 48, 49, 57,  7, 54,  8,  5,
        57, 39,  5, 45, 48, 49, 57,  7, 54, 10,  4,  4, 19, 43, 52, 53, 54,  5,
        16, 43, 54, 43, 60, 39, 48, 11,  4, 23, 39, 54,  5, 55, 53,  5, 45, 43,
        46, 46,  5, 42, 43, 47,  8,  5, 35, 48, 38,  5, 57, 39,  7, 46, 46,  5,
        42, 35, 56, 39,  5, 37, 49, 52, 48,  5, 35, 54,  5, 49, 55

# Model

In [14]:
# Initialize the model
model = GPT(model_config).to(device)
model = torch.compile(model)
print(model)
print(f"Number of parameters: {model.num_params() / 1e6:.2f}M")

OptimizedModule(
  (_orig_mod): GPT(
    (token_embedding): Embedding(61, 128)
    (positional_encoding): Embedding(128, 128)
    (dropout): Dropout(p=0.1, inplace=False)
    (blocks): ModuleList(
      (0-3): 4 x Block(
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (attn): MultiHeadAttention(
          (Wq): Linear(in_features=128, out_features=32, bias=False)
          (Wkv_down): Linear(in_features=128, out_features=32, bias=False)
          (Wk_up): Linear(in_features=32, out_features=128, bias=False)
          (Wv_up): Linear(in_features=32, out_features=128, bias=False)
          (out_proj): Linear(in_features=128, out_features=128, bias=False)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (mlp): FeedForward(
          (fc1): Linear(in_features=128, out_features=512, bias=False)
          (activation): GELU(approximate='none')
          (fc2): Linear(in_f

# Training

In [15]:
trainer = Trainer(
    model=model,
    train_config=train_config,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    master_process=True
)
trainer.train()

Training:   0%|          | 0/9 [00:00<?, ?it/s]E0605 09:35:24.021000 48894 site-packages/torch/_subclasses/fake_tensor.py:2017] [0/0] failed while attempting to run meta for aten.view.default
E0605 09:35:24.021000 48894 site-packages/torch/_subclasses/fake_tensor.py:2017] [0/0] Traceback (most recent call last):
E0605 09:35:24.021000 48894 site-packages/torch/_subclasses/fake_tensor.py:2017] [0/0]   File "/opt/conda/envs/torch-env/lib/python3.12/site-packages/torch/_subclasses/fake_tensor.py", line 2013, in _dispatch_impl
E0605 09:35:24.021000 48894 site-packages/torch/_subclasses/fake_tensor.py:2017] [0/0]     r = func(*args, **kwargs)
E0605 09:35:24.021000 48894 site-packages/torch/_subclasses/fake_tensor.py:2017] [0/0]         ^^^^^^^^^^^^^^^^^^^^^
E0605 09:35:24.021000 48894 site-packages/torch/_subclasses/fake_tensor.py:2017] [0/0]   File "/opt/conda/envs/torch-env/lib/python3.12/site-packages/torch/_ops.py", line 716, in __call__
E0605 09:35:24.021000 48894 site-packages/torch/_s

TorchRuntimeError: Failed running call_method view(*(FakeTensor(..., device='cuda:0', size=(1024, 128, 32), dtype=torch.bfloat16,
           grad_fn=<ViewBackward0>), 1024, 128, 4, 32), **{}):
shape '[1024, 128, 4, 32]' is invalid for input of size 4194304

from user code:
   File "/workspace/PathFinder/models/GPT.py", line 337, in forward
    x, kv_cache_layer = block(x, kv_cache=kv_cache[layer_idx])                  # [batch_size, seq_len, d_embed]
  File "/workspace/PathFinder/models/GPT.py", line 274, in forward
    attn_out, new_kv_cache = self.attn(x, kv_cache=kv_cache)
  File "/workspace/PathFinder/models/GPT.py", line 90, in forward
    q = q.view(batch_size, seq_len, self.config.n_heads, self.config.d_head).transpose(1, 2)

Set TORCH_LOGS="+dynamo" and TORCHDYNAMO_VERBOSE=1 for more information


You can suppress this exception and fall back to eager by setting:
    import torch._dynamo
    torch._dynamo.config.suppress_errors = True


## Save the model

In [ ]:
# Save model locally
if train_config.debug:
    output_dir = os.path.join(PROJECT_ROOT, "checkpoints", train_config.model_name, train_config.run_name)
    os.makedirs(output_dir, exist_ok=True)
    try:
        model.save_pretrained(
            output_dir,
            safe_serialization=True
        )
        print("Model saved successfully.")
    except Exception as e:
        print(f"Error saving model: {e}")
    # Push to Hugging Face Hub
    #model.push_to_hub(
    #    repo_id=f"PathFinderKR/{train_config.model_name}-{train_config.run_name}",
    #    private=True,
    #    use_auth_token=os.environ.get("HUGGINGFACE_TOKEN")
    #)
    #print(f"Model pushed to Hugging Face Hub: PathFinderKR/{train_config.model_name}-{train_config.run_name}")

In [ ]:
# To load the model later, you can use:
# model = GPT(model_config)
# model = model.from_pretrained(output_dir).to(device)

# Inference

In [ ]:
user_prompt = "To be, or not to be, that is the question"
input_ids = char_tokenizer.encode(user_prompt).unsqueeze(0).to(device)
output = model.generate(
    input_ids,
    use_cache=True,
    max_new_tokens=generation_config.max_new_tokens,
    temperature=generation_config.temperature,
    top_k=generation_config.top_k,
    tokenizer=char_tokenizer
)
response = char_tokenizer.decode(output[0].squeeze().cpu().numpy())

In [ ]:
print("=" * 50)
print("User prompt: ")
print(user_prompt)
print("-" * 50)
print("🤖 Model Response:")
print(response)

# Speedometer

In [ ]:
speedometer(
    model=model,
    input_ids=char_tokenizer.encode("a").unsqueeze(0).to(device),
    use_cache=True,
    warmup_tokens=100,
    timing_tokens=100,
    num_runs=5
)

In [ ]:
speedometer(
    model=model,
    input_ids=char_tokenizer.encode("a").unsqueeze(0).to(device),
    use_cache=False,
    warmup_tokens=100,
    timing_tokens=100,
    num_runs=5
)